In [1]:
import ee
import geemap

Map = geemap.Map()

point = ee.Geometry.Point([106.9942, -6.2764])


image = ee.Image('LANDSAT/LC08/C01/T1_TOA/LC08_122064_20200422')
Map.addLayer(
    image,
    {'bands': ['B4',  'B3',  'B2'], 'min': 0, 'max': 0.25, 'gamma': [1.1, 1.1, 1]},
    'rgb')



#Map.centerObject(image,8)

#Map

In [3]:
developed1 = ee.Geometry.Point([106.9860, -6.2144]).buffer(100)
developed2 = ee.Geometry.Point([106.9877, -6.2143]).buffer(100)
developed3 = ee.Geometry.Point([106.9801, -6.2209]).buffer(100)
developed4 = ee.Geometry.Point([106.98, -6.24]).buffer(100)
developed5 = ee.Geometry.Point([106.99, -6.24]).buffer(100)
water1 = ee.Geometry.Point([106.9836, -6.2749]).buffer(10)
water2 = ee.Geometry.Point([106.9794, -6.2770]).buffer(10)
water3 = ee.Geometry.Point([106.9751, -6.2905]).buffer(10)
water4 = ee.Geometry.Point([106.9751, -6.2905]).buffer(10)
water5 = ee.Geometry.Point([106.9722, -6.2951]).buffer(10)
open_field1 = ee.Geometry.Point([106.9772, -6.3021]).buffer(100)
open_field2 = ee.Geometry.Point([106.9528, -6.2816]).buffer(100)
open_field3 = ee.Geometry.Point([106.947,-6.2997]).buffer(100)
open_field4 = ee.Geometry.Point([106.9536,-6.2937]).buffer(100)
open_field5 = ee.Geometry.Point([106.9552,-6.3421]).buffer(100)

#Jadiin satu data training
points = ee.FeatureCollection([
  ee.Feature(developed1, {'landcover': 0}),
  ee.Feature(developed2, {'landcover': 0}),
  ee.Feature(developed3, {'landcover': 0}),
  ee.Feature(developed4, {'landcover': 0}),
  ee.Feature(developed5, {'landcover': 0}),
  ee.Feature(water1, {'landcover': 1}),
  ee.Feature(water2, {'landcover': 1}),
  ee.Feature(water3, {'landcover': 1}),
  ee.Feature(water4, {'landcover': 1}),
  ee.Feature(water5, {'landcover': 1}),  
  ee.Feature(open_field1, {'landcover': 2}),
  ee.Feature(open_field2, {'landcover': 2}),
  ee.Feature(open_field3, {'landcover': 2}),
  ee.Feature(open_field4, {'landcover': 2}),
  ee.Feature(open_field5, {'landcover': 2}),
])

#Map.addLayer(points, {}, 'training', False)

#Map.centerObject(kota_bekasi,12)

#Map

In [4]:
# Use these bands for prediction.
bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7']


# This property of the table stores the land cover labels.
label = 'landcover'

# Overlay the points on the imagery to get training.
training = image.select(bands).sampleRegions(**{
  'collection': points,
  'properties': [label],
  'scale': 30
})

# Train a CART classifier with default parameters.
trained = ee.Classifier.smileCart().train(training, label, bands)


In [5]:
print(training.first().getInfo())

{'type': 'Feature', 'geometry': None, 'id': '0_0', 'properties': {'B1': 0.15847483277320862, 'B2': 0.13851679861545563, 'B3': 0.1245771050453186, 'B4': 0.1351151317358017, 'B5': 0.20752543210983276, 'B6': 0.2366180121898651, 'B7': 0.18911361694335938, 'landcover': 0}}


In [6]:
# Classify the image with the same bands used for training.
result = image.select(bands).classify(trained)

# # Display the clusters with random colors.
#Map.addLayer(result.randomVisualizer(), {}, 'classified')
#Map

In [7]:
kota_bekasi_klasifikasi = result.clip(cities.geometry())
#Map.addLayer(kota_bekasi_klasifikasi.randomVisualizer(), {}, 'Bekasi City Classified')
#Map

In [8]:
legend_keys = ['Developed', 'Open Field', 'Water']
legend_colors = ['#eb6b34', '#23751a', '#6534eb']

# Reclassify the map
result = kota_bekasi_klasifikasi.remap([0, 1, 2], [1, 2, 3])

Map.addLayer(kota_bekasi_klasifikasi, {'min': 1, 'max': 5, 'palette': legend_colors}, 'Landcover Bekasi')
Map.add_legend(legend_keys=legend_keys, legend_colors=legend_colors, position='bottomright', 
               color='#e4ede6')


Map.centerObject(kota_bekasi_klasifikasi,12)
Map


Map(center=[-6.246151075999909, 107.04148116900006], controls=(WidgetControl(options=['position', 'transparent…

In [9]:
import os

In [10]:
download_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
if not os.path.exists(download_dir):
    os.makedirs(download_dir)
html_file = os.path.join(download_dir, 'my_map.html')

In [11]:
Map.to_html(outfile=html_file, title='My Map', width='100%', height='880px')